# Taller 1 - DATASETS Y TOKENIZADORES

Realizado por:
* Ortiz Rodríguez Nicol valeria
* Tovar Parra Carlos Mauricio
* Camargo Arcos Andres David


## Segunda Parte (40 puntos)

In [1]:
!pip install datasets transformers tokenizers
from datasets import load_dataset
import os

INFO: pip is looking at multiple versions of multiprocess to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 471.6/471.6 kB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 12.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 16.7 MB/s eta 0:00:00


**Cargar el dataset JRC desde Hugging Face**

In [2]:
dataset = load_dataset('large_spanish_corpus', name='JRC')['train']

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


large_spanish_corpus.py:   0%|          | 0.00/3.93k [00:00<?, ?B/s]

README.md:   0%|          | 0.00/8.26k [00:00<?, ?B/s]

The repository for large_spanish_corpus contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/large_spanish_corpus.
You can avoid this prompt in future by passing the argument `trust_remote_code=True`.

Do you wish to run the custom code? [y/N] y


raw.zip:   0%|          | 0.00/5.79G [00:00<?, ?B/s]

Generating train split:   0%|          | 0/3410620 [00:00<?, ? examples/s]

In [3]:
print(dataset)

Dataset({
    features: ['text'],
    num_rows: 3410620
})


**Separar dataset en varios archivos**

In [5]:
os.makedirs("JRC-dataset", exist_ok=True)

def save_text_files(dataset, num_files=10):
    total_rows = len(dataset)
    rows_per_file = total_rows // num_files
    for i in range(num_files):
        with open(f"JRC-dataset/text_file_{i}.txt", "w", encoding="utf-8") as f:
            start = i * rows_per_file
            end = (i + 1) * rows_per_file if i < num_files - 1 else total_rows
            for j in range(start, end):
                text = dataset[j]['text']
                f.write(text + '\n')

save_text_files(dataset)

**Inicializa el tokenizador**

In [6]:
from tokenizers import BertWordPieceTokenizer
tokenizer = BertWordPieceTokenizer(
    clean_text=True,
    handle_chinese_chars=False,
    strip_accents=False,
    lowercase=False
)

**Entrena el tokenizador con los archivos de texto generados**

In [8]:
tokenizer.train(files=[f"JRC-dataset/text_file_{i}.txt" for i in range(10)], vocab_size=30000, min_frequency=2,
                limit_alphabet=1000, wordpieces_prefix='##',
                special_tokens=[
                    '[PAD]', '[UNK]', '[CLS]', '[SEP]', '[MASK]'
                ])

In [9]:
try:
    os.makedirs('./bert22')
except FileExistsError:
    print('Directory already exists')

In [10]:
tokenizer.save_model('./bert22/', 'bert-it')

['./bert22/bert-it-vocab.txt']

In [11]:
from transformers import BertTokenizer

The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


0it [00:00, ?it/s]

In [12]:
tokenizer = BertTokenizer.from_pretrained('./bert22/bert-it-vocab.txt')

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:2165: FutureWarning: Calling BertTokenizer.from_pretrained() with the path to a single file or url is deprecated and won't be possible anymore in v5. Use a model identifier or the path to a directory instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


**Se tokeniza una oración de prueba**

In [19]:
sentence = "Todos los bachilleres estudian en Univalle"
tokens = tokenizer(sentence)
print("Tokens:", tokenizer.tokenize(sentence))

Tokens: ['todos', 'los', 'bac', '##hi', '##ll', '##eres', 'estudi', '##an', 'en', 'un', '##ival', '##le']


**Se prueba el transformer original de bert-base-uncased**

In [20]:
from transformers import AutoTokenizer, AutoModelForMaskedLM

autotokenizer = AutoTokenizer.from_pretrained("google-bert/bert-base-uncased")

In [21]:
a_tokens = autotokenizer(sentence)
print("Tokens:", autotokenizer.tokenize(sentence))

Tokens: ['tod', '##os', 'los', 'bach', '##ille', '##res', 'est', '##udi', '##an', 'en', 'un', '##iva', '##lle']
